In [1]:
from gensim import models

word2vec_path = './GoogleNews-vectors-negative300.bin.gz'
model = models.keyedvectors.load_word2vec_format(word2vec_path, binary=True)

In [2]:
import plotly
import numpy as np
import plotly.graph_objs as go
from sklearn.decomposition import PCA

In [17]:
model.most_similar('target', 10)

[('targets', 0.5353715419769287),
 ('targeting', 0.4455298185348511),
 ('targeted', 0.4379848837852478),
 ('targetting', 0.4110974371433258),
 ('#,####_+##', 0.4028463363647461),
 ('#,####_+###_pips', 0.38023340702056885),
 ('###,##_+###_pips', 0.3609311878681183),
 ('targetted', 0.35414013266563416),
 ('pips_#,####_+###_pips', 0.3514072000980377),
 ('risk_reward_#:#.#', 0.33798882365226746)]

In [22]:
def append_list(sim_words, words):

    list_of_words = []

    for i in range(len(sim_words)):

        sim_words_list = list(sim_words[i])
        sim_words_list.append(words)
        sim_words_tuple = tuple(sim_words_list)
        list_of_words.append(sim_words_tuple)

    return list_of_words

input_word = 'Ukraine, Russia, sanction, Crimea, Donbass'
user_input = [x.strip() for x in input_word.split(',')]
result_word = []

for words in user_input:

        sim_words = model.most_similar(words, topn = 5)
        sim_words = append_list(sim_words, words)

        result_word.extend(sim_words)

similar_word = [word[0] for word in result_word]
similarity = [word[1] for word in result_word]
similar_word.extend(user_input)
labels = [word[2] for word in result_word]
label_dict = dict([(y,x+1) for x,y in enumerate(set(labels))])
color_map = [label_dict[x] for x in labels]

In [23]:
input_word2 = 'Kiev, drone, missile, army, target'
user_input2 = [x.strip() for x in input_word2.split(',')]
result_word2 = []

for words in user_input2:

        sim_words = model.most_similar(words, topn = 5)
        sim_words = append_list(sim_words, words)

        result_word2.extend(sim_words)

similar_word2 = [word[0] for word in result_word2]
similarity2 = [word[1] for word in result_word2]
similar_word2.extend(user_input)
labels2 = [word[2] for word in result_word2]
label_dict2 = dict([(y,x+1) for x,y in enumerate(set(labels))])
color_map2 = [label_dict[x] for x in labels]

In [24]:
def display_pca_scatterplot_3D(model, user_input=None, words=None, label=None, color_map=None, topn=5, sample=10):

    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.vocab.keys()), sample)
        else:
            words = [ word for word in model.vocab ]

    word_vectors = np.array([model[w] for w in words])

    three_dim = PCA(random_state=0).fit_transform(word_vectors)[:,:3]
    # For 2D, change the three_dim variable into something like two_dim like the following:
    # two_dim = PCA(random_state=0).fit_transform(word_vectors)[:,:2]

    data = []
    count = 0

    for i in range (len(user_input)):

                trace = go.Scatter3d(
                    x = three_dim[count:count+topn,0],
                    y = three_dim[count:count+topn,1],
                    z = three_dim[count:count+topn,2],
                    text = words[count:count+topn],
                    name = user_input[i],
                    textposition = "top center",
                    textfont_size = 20,
                    mode = 'markers+text',
                    marker = {
                        'size': 10,
                        'opacity': 0.8,
                        'color': 2
                    }

                )

                # For 2D, instead of using go.Scatter3d, we need to use go.Scatter and delete the z variable. Also, instead of using
                # variable three_dim, use the variable that we have declared earlier (e.g two_dim)

                data.append(trace)
                count = count+topn

    trace_input = go.Scatter3d(
                    x = three_dim[count:,0],
                    y = three_dim[count:,1],
                    z = three_dim[count:,2],
                    text = words[count:],
                    name = 'input words',
                    textposition = "top center",
                    textfont_size = 20,
                    mode = 'markers+text',
                    marker = {
                        'size': 10,
                        'opacity': 1,
                        'color': 'grey'
                    }
                    )

    # For 2D, instead of using go.Scatter3d, we need to use go.Scatter and delete the z variable.  Also, instead of using
    # variable three_dim, use the variable that we have declared earlier (e.g two_dim)

    data.append(trace_input)

# Configure the layout

    layout = go.Layout(
        margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
        showlegend=True,
        legend=dict(
        x=1,
        y=0.5,
        font=dict(
            family="Courier New",
            size=25,
            color="white"
        )),
        font = dict(
            family = " Courier New ",
            size = 15),
        autosize = False,
        width = 1000,
        height = 1000
        )


    plot_figure = go.Figure(data = data, layout = layout)
    plot_figure.show()

display_pca_scatterplot_3D(model, user_input, similar_word, labels, color_map)
display_pca_scatterplot_3D(model, user_input2, similar_word2, labels2, color_map2)